In [2]:
!pip3 install pyspark


     |████████████████████████████████| 212.4MB 62kB/s 
     |████████████████████████████████| 204kB 15.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=0aa1cab60c97a8c8f1f1cf52c551be6563e4ac41d35d4add30c41452ffed1757
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [4]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['spark-3.1.2']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$spark_version/$spark_version-bin-hadoop2.7.tgz
!tar xf $spark_version-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [5]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [6]:
#import Tokenizer library
from pyspark.ml.feature import Tokenizer


In [7]:
#create sample dataframe
dataframe = spark.createDataFrame([
    (0, "Spark is great"),
    (1, "We are learning Spark"),
    (2, "Spark is better than hadoop no doubt")
], ["id", "sentence"])                              
      
dataframe.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|      Spark is great|
|  1|We are learning S...|
|  2|Spark is better t...|
+---+--------------------+



In [8]:
# Tokenize sentences
#input passes name of column that we want to be tokenized
#output takes the name that we want the column to be called
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_a3cf0f2cf8f2

In [9]:
#transform and show dataframe
#transform method takes the df as input, which is a transformation
tokenized_df = tokenizer.transform(dataframe)
tokenized_df.show(truncate=False)
#user truncate=false to display the results without shortening the output

+---+------------------------------------+--------------------------------------------+
|id |sentence                            |words                                       |
+---+------------------------------------+--------------------------------------------+
|0  |Spark is great                      |[spark, is, great]                          |
|1  |We are learning Spark               |[we, are, learning, spark]                  |
|2  |Spark is better than hadoop no doubt|[spark, is, better, than, hadoop, no, doubt]|
+---+------------------------------------+--------------------------------------------+



In [15]:
# Create a function to return the length of a list
#takes list of words as its input, then returns the length of that list
def word_list_length(word_list):
    return len(word_list)

In [16]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [17]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [18]:
#create our tokenizer
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

#transform dataframe
tokenized_df = tokenizer.transform(dataframe)

#select the needed columns and dont truncate results
tokenized_df.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

#redo tokenizer process. After the df has outputted the tokenized values, we can use our own created function to return the number of tokens created


+---+------------------------------------+--------------------------------------------+------+
|id |sentence                            |words                                       |tokens|
+---+------------------------------------+--------------------------------------------+------+
|0  |Spark is great                      |[spark, is, great]                          |3     |
|1  |We are learning Spark               |[we, are, learning, spark]                  |4     |
|2  |Spark is better than hadoop no doubt|[spark, is, better, than, hadoop, no, doubt]|7     |
+---+------------------------------------+--------------------------------------------+------+

